# Web-scraping: сбор данных из баз данных и интернет-источников

*Алла Тамбовцева*

## Регулярные выражения: обзор возможностей

Регулярные выражения – выражения, последовательности символов, которые позволяют искать совпадения в тексте. Выражаясь более формально, они помогают найти подстроки определенного вида в строке. Еще о регулярных выражениях можно думать как о шаблонах, в которые мы можем подставлять текст, и этот текст либо соответствует шаблону, либо нет.

В самом простом случае в качестве регулярного выражения может использоваться обычная строка. Например, чтобы проверить, есть ли в предложении *Кошка сидит под столом.* слово *Кошка*, ничего специального применять не нужно, достаточно воспользоваться оператором `in`. Если нас интересует слово *кошка* в любом регистре, то это уже более интересная задача. Правда, ее все еще можно решить без регулярных выражений, приведя все слова в строке к нижнему регистру. А что, если у нас будет текст подлиннее, и в нем необходимо «обнаружить» кошку в разных падежах? И еще производные слова вроде *кошечка*? Тут уже удобнее написать некоторый шаблон, чтобы не создавать длинный список слов с разными формами слова *кошка*. Давайте немного потренируемся (но не на кошках).

Импортируем модуль `re` для работы с регулярными выражениями:

In [1]:
import re

### Сюжет 1. Функция `findall()` и поиск слов

В качестве игрушечного примера возьмем обычную строку с немного странным текстом:

In [2]:
words = "дом дым дума Дуся думать Дульсинея Дина"

Найдем в этой строке все подстроки, которые содержат букву `д`. Для этого нам понадобится функция `findall()`:

In [3]:
# сначала шаблон (pattern)
# затем – сама строка, где ищем

re.findall("д", words)

['д', 'д', 'д', 'д']

Функция `findall()` возвращает список найденных строк ровно того вида, который мы задали регулярным выражением. Теперь попробуем найти слова *дом* и *дым*. Так как первая буква и последняя одинаковы, можем написать шаблон, где между ними подставляется *о* или *ы*. Множество символов, которые могут встретиться (или один, или другой) на определенном месте указывается в квадратных скобках:

In [4]:
# множество [оы]
re.findall("д[оы]м", words)

['дом', 'дым']

Если привычнее, можно между буквами поставить `|` – как оператор «или» в Python:

In [5]:
# то же самое
re.findall("д[о|ы]м", words)

['дом', 'дым']

Если мы допускаем, что между буквами может стоять любой символ (в данном случае кроме букв и пробелов в строке все равно ничего нет), его можно обозначить с помощью точки:

In [6]:
# . – любой символ
re.findall("д.м", words)

['дом', 'дым', 'дум', 'дум']

Если слова *дом* и *дым* нам не нужны, а мы хотим забрать только слова, однокоренные с глаголом «думать», нужно написать выражение, которое будет учитывать, что после `д.м` есть еще буква. Это можно сделать с помощью уже знакомого множества с квадратными скобками, только внутри указать интервал:

In [7]:
# добавляем любую букву от а-я
# (именно маленькую)

re.findall("д.м[а-я]", words)

['дума', 'дума']

В квадратные скобки можно поместить готовые интервалы символов:

* `[a-z]`: строчные буквы английского алфавита;
* `[A-Z]`: заглавные буквы английского алфавита;
* `[а-я]`: строчные буквы русского алфавита;
* `[А-Я]`: заглавные буквы русского алфавита;
* `[0-9]`: цифры от 0 до 9.

Если мы допускаем буквы и заглавные, и строчные, в скобках можно указать оба интервала:

In [8]:
re.findall("д.м[а-яА-Я]", words)

['дума', 'дума']

Второе слово обрезано – потому что пока буква в выражении указана строго одна. Если нас интересуют не фрагменты слов, а слова полностью, нужно учесть, что буква может быть не одна. В регулярных выражениях есть следующие полезные операторы:

* `*`: символ слева от `*` встречается 0 и более раз;
* `?`: символ слева от `?` встречается 0 или 1 раз;
* `+`: символ слева от `+` встречается 1 и более раз.

Сравним результаты:

In [9]:
# д.м, далее 0 или более букв
# варианты, где после м букв нет, и где они есть

re.findall("д.м[а-я]*", words)

['дом', 'дым', 'дума', 'думать']

In [10]:
# д.м, далее 0 или 1 буква
# местами обрезанные слова, раз 1 буква после м – это максимум

re.findall("д.м[а-я]?", words) # ? 0 или 1 раз

['дом', 'дым', 'дума', 'дума']

In [11]:
# д.м, далее 1 или более букв
# только варианты с дума и думать

re.findall("д.м[а-я]+", words) # + 1 и более раз

['дума', 'думать']

Если мы захотим забрать только имена, нужно будет учитывать слова с заглавной буквы:

In [12]:
# любая заглавная, затем сколько угодно маленьких
# (считаем, что имен из одной буквы здесь нет,
# если бы были, то тогда в конце нужна * вместо +)

re.findall("[А-Я][а-я]+", words)

['Дуся', 'Дульсинея', 'Дина']

И последний пример для этой строки – имена, которые начинаются на *Д* и заканчиваются на *я*:

In [13]:
re.findall("Д[а-я]+я", words)

['Дуся', 'Дульсинея']

### Сюжет 2. Функция `split()` и работа с предложениями

Создадим строку с уже более поэтичным маленьким текстом из трех предложений:

In [14]:
sentence = "И пришла зима. Начались морозы. Инеем все покрылось."

Попробуем извлечь все слова из текста:

In [15]:
# теряем заглавные буквы
re.findall("[а-я]+", sentence)

['пришла', 'зима', 'ачались', 'морозы', 'неем', 'все', 'покрылось']

In [16]:
# ничего не теряем
re.findall("[а-яА-Я]+", sentence)

['И', 'пришла', 'зима', 'Начались', 'морозы', 'Инеем', 'все', 'покрылось']

In [17]:
# тоже вариант – ничего не теряем
re.findall("[А-я]+", sentence)

['И', 'пришла', 'зима', 'Начались', 'морозы', 'Инеем', 'все', 'покрылось']

Также учесть регистр можно, добавив аргумент `flags`:

In [18]:
# флаги – дополнительные настройки
# IGNORECASE – игнорируем регистр

re.findall("[а-я]+", sentence, flags = re.IGNORECASE)

['И', 'пришла', 'зима', 'Начались', 'морозы', 'Инеем', 'все', 'покрылось']

Если нам для какой-то цели нужно будет работать со знаками препинания, точку нужно будет экранировать – поставить перед ней `\` или `\\`, чтобы четко разделить точку как специальный оператор (`.` – один любой символ или `\.` как знак препинания):

In [19]:
# находим все точки
re.findall("\.", sentence)

['.', '.', '.']

То же будет актуально и для других символов, которые используются в качестве операторов в регулярных выражениях (квадратные скобки, фигурные скобки, круглые скобки, `+`, `*`, `?` и еще ряд других). 

Попробуем использовать регулярное выражение для разбиения текста `sentence` на предложения. Если использовать обычный метод `.split()` на строках, последняя точка тоже будет использована для разбиения, и мы получим лишнюю пустую строку в конце:

In [20]:
sentence.split(".")

['И пришла зима', ' Начались морозы', ' Инеем все покрылось', '']

Исправим это – воспользуемся функцией `split()` из `re` и укажем в выражении, что для разумного деления на предложения нужно, чтобы после точки обязательно стоял пробел (`\s` от *space*):

In [21]:
re.split("\.\s", sentence)

['И пришла зима', 'Начались морозы', 'Инеем все покрылось.']

Тут возникает другая проблема: последняя точка приклеилась к словам :) Но оставим пока ее на месте (тут сложно), и посмотрим на другой вариант текста, где точки используются и еще для сокращений – после инициалов:

In [22]:
sentence2 = "И пришла зима. Начались морозы. Инеем все покрылось! А.Неизвестный"

Разрешим разбивать текст по точкам и восклицательным знакам, если после тех стоит пробел:

In [23]:
re.split("[\.!]\s", sentence2)

['И пришла зима', 'Начались морозы', 'Инеем все покрылось', 'А.Неизвестный']

**Дополнение 1.** В Python есть базовый модуль `string`, внутри которого есть набор основных символов для пунктуации и не только (но не все, нет кавычек-елочек для русского языка и разных видов тире):

In [24]:
import string
print(string.punctuation)

!"#$%&'()*+,-./:;<=>?@[\]^_`{|}~


**Дополнение 2.** Конечно, если речь идет о последующем анализе текста, где необходимо корректное разбиение на слова, обычно пользуются не самостоятельно написанными выражениями, а готовыми библиотеками/модулями с **токенизаторами** для разных языков (токенизатор, англ. *tokenizer* – программа, разбивающая текст на слова, слова в рамках обработки текста называются токенами).

### Сюжет 3. Поиск цифр и чисел

Перейдем к поиску цифр и чисел. Возьмем небольшой текст, где есть даты и некоторые номера.

In [25]:
test = """11 февраля 2025 года ничего особенного не произошло. В 2024 году было то же самое. 
Обычное самое интересное происходит во 2-м полугодии, а не в 1-м.
"""

Для поиска цифр можно записать интервал:

In [26]:
re.findall("[0-9]", test)

['1', '1', '2', '0', '2', '5', '2', '0', '2', '4', '2', '1']

А можно воспользоваться специальным обозначением `\d` (от англ. *digit*, экранируем, чтобы не путать с обычной буквой *d*):

In [27]:
re.findall("\d", test)

['1', '1', '2', '0', '2', '5', '2', '0', '2', '4', '2', '1']

Если нам нужны числа, а не отдельные цифры, нужно учесть, что число – это одна цифра и более:

In [28]:
re.findall("\d+", test)

['11', '2025', '2024', '2', '1']

Если нас интересуют годы, которые здесь четырехзначные, укажем число символов в фигурных скобках:

In [29]:
re.findall("\d{4}", test)

['2025', '2024']

Если нас интересуют только даты в виде номера дня, можем поставить отрицание через `^` и сообщить, что перед первой цифрой других цифр быть не должно:

In [30]:
# 1-2-значные числа \d{1,2}
# избегаем цифры в начале ^\d
re.findall("^\d\d{1,2}", test)

['11']

Если это отрицание не добавим, вдобавок получим пары цифр, входящие в состав года:

In [31]:
re.findall("\d{1,2}", test)

['11', '20', '25', '20', '24', '2', '1']

Совместим поиск текста и чисел. Возьмем строку с реальным [текстом](http://dolcevitablog.ru/2021/10/04/pasta-arrabbiata/) рецепта из блога:

In [32]:
text = """
Злая паста Arrabbiata
Яркий кисло-сладкий вкус томатов с нотками чеснока 
и свежей петрушки через несколько секунд сменяется 
остротой красного перца. Тепло разливается по всем телу… 
именно поэтому мне так нравится готовить аррабиату 
в холодное время года — она согревает:)
Arrabbiato — (ит.) «злой, сердитый», поэтому 
неудивительно, что главными героями в этом рецепте будет 
свежий чеснок и острый перчик. Я люблю итальянский 
пеперончино, но подойдет любой красный на ваш вкус, 
например, чили. Можно даже в сушеный, но со свежим 
вкус и аромат будут просто wow!
Традиционно для аррабиаты используют пенне ригате, 
это такая трубчатая паста. Благодаря своей форме 
и ребристой поверхности она отлично удерживает 
соус и не дает ему стекать на дно тарелки. Конечно 
можно приготовить и с другими схожими видами: 
ригатони, казаречче, тортильони… Но все не просто 
так у этих итальянцев!
Паста для сильных духом! Быстрая, простая, вкусная, 
и с минимальным количеством ингредиентов!
Поехали!
1. Поставьте на огонь большую кастрюлю воды. Посолите 
когда закипит и забросьте пасту. Варить ее нужно будет на 
1 минуту меньше, чем указано на упаковке. Если готовите 
аррабиату в первый раз, то можно сперва приготовить соус, 
а потом сварить пасту — так она точно не разварится.
2. В еще холодную сковороду налейте оливковое масло, 
и отправьте туда нарубленный дольками чеснок и острый 
перец. Только вам решать, насколько «злой» будет ваша паста:) 
Если хочется помягче, удалите из чили семена и перегородки, 
а если поострее — нарубите весь стручок целиком
3. Поставьте на огонь. Когда масло начнет шипеть, добавьте 
мятые томаты или пассату, и тушите соус на среднем огне, 
пока паста не будет готова. Если видите, что соус уже 
загустел, а паста не сварилась, плесните в сковороду 
полполовника воды из кастрюли
4. Попробуйте соус и при необходимости посолите
5. Отчерпните кружкой немного воды от пасты, на всякий 
случай.
6. Слейте пасту, переложите ее в соус. Подержите 
на огне, помешивая, еще минуту-две, пока паста 
не дойдет до готовности. Отрегулируйте консистенцию 
водой от пасты
7. Выключите огонь и добавьте рубленную петрушку
"""

Попробуем посчитать, сколько раз пасту назвали злой :) Сердитые макароны – это серьезно:

In [33]:
# попытка 1: не все формы слова
re.findall("[Зз]лая", text)

['Злая']

In [34]:
# попытка 2: больше форм слова
re.findall("[Зз]л[ая|ой]+", text)

['Злая', 'злой', 'злой']

In [35]:
# попытка 3: все формы слова + лишний фрагмент
re.findall("[Зз]л[а-я]+", text)

['Злая', 'зливается', 'злой', 'злой']

Чтобы избавиться от лишнего фрагмента *зливается*, добавим перед первой буквой отрицание к символам, которые могут входит в состав слова (`\w` – это любой символ для «слова», включает буквы, цифры и подчеркивание `_`, `\W` – наоборот, все, что к этим символам не относится):

In [36]:
# ловятся варианты с кавычками
# и первое слово, где в начале текста перед на новую строку \n
re.findall("\W[Зз]л[а-я]+", text)

['\nЗлая', '«злой', '«злой']

Получается, три раза. 

Извлечем непосредственно сами шаги приготовления блюда – фрагменты, которые начинаются с числа с точкой (номер пункта):

In [37]:
re.findall("\d\..+", text)

['1. Поставьте на огонь большую кастрюлю воды. Посолите ',
 '2. В еще холодную сковороду налейте оливковое масло, ',
 '3. Поставьте на огонь. Когда масло начнет шипеть, добавьте ',
 '4. Попробуйте соус и при необходимости посолите',
 '5. Отчерпните кружкой немного воды от пасты, на всякий ',
 '6. Слейте пасту, переложите ее в соус. Подержите ',
 '7. Выключите огонь и добавьте рубленную петрушку']

Проблема: один пункт не помещается в одной строке, текст обрезается. Логично использовать другой подход – умное разбиение через `split()`: 

In [38]:
L = re.split("\d\.", text)

Теперь первый элемент в `L` – это описание рецепта, а все остальные – шаги приготовления (пункты):

In [40]:
descr = L[0]
print(descr)


Злая паста Arrabbiata
Яркий кисло-сладкий вкус томатов с нотками чеснока 
и свежей петрушки через несколько секунд сменяется 
остротой красного перца. Тепло разливается по всем телу… 
именно поэтому мне так нравится готовить аррабиату 
в холодное время года — она согревает:)
Arrabbiato — (ит.) «злой, сердитый», поэтому 
неудивительно, что главными героями в этом рецепте будет 
свежий чеснок и острый перчик. Я люблю итальянский 
пеперончино, но подойдет любой красный на ваш вкус, 
например, чили. Можно даже в сушеный, но со свежим 
вкус и аромат будут просто wow!
Традиционно для аррабиаты используют пенне ригате, 
это такая трубчатая паста. Благодаря своей форме 
и ребристой поверхности она отлично удерживает 
соус и не дает ему стекать на дно тарелки. Конечно 
можно приготовить и с другими схожими видами: 
ригатони, казаречче, тортильони… Но все не просто 
так у этих итальянцев!
Паста для сильных духом! Быстрая, простая, вкусная, 
и с минимальным количеством ингредиентов!
Поехали!



In [41]:
steps = L[1:]
print(len(steps)) # все 7 шагов, отдельные абзацы в списке

7


### Сюжет 4. Функция `search()` и группы 

Перейдем от работы с отдельными строками к списку строк. Это прогнозы погоды в Сочи и Москве в ближайшие дни:

In [42]:
pogoda = [
    "вчера, 11 февраля, небольшой дождь, днём 10°, ночью 2°",
    "сегодня, 12 февраля, малооблачно, днём 10°, ночью 4°",
    "четверг, 13 февраля, облачно с прояснениями, днём 10°, ночью 5°",
    "пятница, 14 февраля, пасмурно, днём 7°, ночью 5°", 
    "суббота, 15 февраля, ливень, днём 8°, ночью 6°",
    "сегодня, 11 февраля, пасмурно, днём -4°, ночью -6°", 
    "среда, 12 февраля, пасмурно, днём -4°, ночью -6°", 
    "четверг, 13 февраля, пасмурно, днём -5°, ночью -5°", 
    "пятница, 14 февраля, небольшой снег, днём -4°, ночью -6°",
    "суббота, 15 февраля, небольшой снег, днём -6°, ночью -10°", 
    "воскресенье, 16 февраля, небольшой снег, днём -8°, ночью -11°"
]

Представим, что мы хотим забрать только числовые данные. Применим функцию `findall()` в цикле и получим список списков:

In [43]:
[re.findall("\d+", p) for p in pogoda]

[['11', '10', '2'],
 ['12', '10', '4'],
 ['13', '10', '5'],
 ['14', '7', '5'],
 ['15', '8', '6'],
 ['11', '4', '6'],
 ['12', '4', '6'],
 ['13', '5', '5'],
 ['14', '4', '6'],
 ['15', '6', '10'],
 ['16', '8', '11']]

Есть номер месяца, есть температура, но в температуре потеряны минусы (где они есть). Напишем выражение, которое эти минусы учтет:

In [44]:
[re.findall("-?\d+", p) for p in pogoda]

[['11', '10', '2'],
 ['12', '10', '4'],
 ['13', '10', '5'],
 ['14', '7', '5'],
 ['15', '8', '6'],
 ['11', '-4', '-6'],
 ['12', '-4', '-6'],
 ['13', '-5', '-5'],
 ['14', '-4', '-6'],
 ['15', '-6', '-10'],
 ['16', '-8', '-11']]

Если нас интересует только температура, добавим символ для градуса:

In [45]:
[re.findall("-?\d+°", p) for p in pogoda]

[['10°', '2°'],
 ['10°', '4°'],
 ['10°', '5°'],
 ['7°', '5°'],
 ['8°', '6°'],
 ['-4°', '-6°'],
 ['-4°', '-6°'],
 ['-5°', '-5°'],
 ['-4°', '-6°'],
 ['-6°', '-10°'],
 ['-8°', '-11°']]

Теперь поставим более глобальную задачу: забрать из текста полную дату с названием месяца и температуру воздуха днем и ночью. Чтобы это сделать, нам понадобится сгруппировать символы внутри строки (группа для даты и группа для температуры ночью и днем. Познакомимся с функцией `search()`:

In [46]:
[re.search("-?\d+°", p) for p in pogoda]

[<re.Match object; span=(41, 44), match='10°'>,
 <re.Match object; span=(39, 42), match='10°'>,
 <re.Match object; span=(50, 53), match='10°'>,
 <re.Match object; span=(36, 38), match='7°'>,
 <re.Match object; span=(34, 36), match='8°'>,
 <re.Match object; span=(36, 39), match='-4°'>,
 <re.Match object; span=(34, 37), match='-4°'>,
 <re.Match object; span=(36, 39), match='-5°'>,
 <re.Match object; span=(42, 45), match='-4°'>,
 <re.Match object; span=(42, 45), match='-6°'>,
 <re.Match object; span=(46, 49), match='-8°'>]

Функция `search()` находит только первое совпадение (по умолчанию) и возвращает не просто подстроки, которые соответствуют выражению, а объекты специального типа `Match`. Из такого объекта помимо самой подстроки можно получить информацию о положении подстроки в исходном тексте (числовые индексы в `span` выше) и группы символов, если деление на группы предусмотрено внутри регулярно выражения. 

В нашем случае пока никакого деления на группы не было, поэтому группа здесь одна единственная – температура воздуха днем (первое совпадение, температура ночью уже не попадает):

In [47]:
[re.search("-?\d+°", p).group() for p in pogoda]

['10°', '10°', '10°', '7°', '8°', '-4°', '-4°', '-5°', '-4°', '-6°', '-8°']

Попробуем теперь написать подобное выражение для даты:

In [48]:
# число (одна цифра и более), пробел \s,
# затем набор букв

[re.search("\d+\s[а-я]+", p).group() for p in pogoda]

['11 февраля',
 '12 февраля',
 '13 февраля',
 '14 февраля',
 '15 февраля',
 '11 февраля',
 '12 февраля',
 '13 февраля',
 '14 февраля',
 '15 февраля',
 '16 февраля']

Попробуем в выражение выше добавить группы – отделить день от месяца, заключив число и текст в круглые скобки:

In [49]:
# первая группа: число (\d+)
# пробел нас не интересует, он просто есть, без скобок
# вторая группа: текст ([а-я]+)

[re.search("(\d+)\s([а-я]+)", p).groups() for p in pogoda]

[('11', 'февраля'),
 ('12', 'февраля'),
 ('13', 'февраля'),
 ('14', 'февраля'),
 ('15', 'февраля'),
 ('11', 'февраля'),
 ('12', 'февраля'),
 ('13', 'февраля'),
 ('14', 'февраля'),
 ('15', 'февраля'),
 ('16', 'февраля')]

Теперь найденный текст разбит на нужные нам части, при этом ненужные части (которые не выделялись скобками) в итоговый результат не попали. Напишем более серьезное выражение, которое извлечет дату и температуру:

In [50]:
[re.search("(\d+)\s([а-я]+).+\s(-?\d+°).+\s(-?\d+°)", p).groups() for p in pogoda]

[('11', 'февраля', '10°', '2°'),
 ('12', 'февраля', '10°', '4°'),
 ('13', 'февраля', '10°', '5°'),
 ('14', 'февраля', '7°', '5°'),
 ('15', 'февраля', '8°', '6°'),
 ('11', 'февраля', '-4°', '-6°'),
 ('12', 'февраля', '-4°', '-6°'),
 ('13', 'февраля', '-5°', '-5°'),
 ('14', 'февраля', '-4°', '-6°'),
 ('15', 'февраля', '-6°', '-10°'),
 ('16', 'февраля', '-8°', '-11°')]

Тут явно требуются пояснения :) Разберем выражение на части.
    
* `(\d+)\s([а-я]+)`: группы для дня и месяца, уже знаем;
* `.+`: после дня и месяца может быть сколько угодно любых символов (текст о погоде с запятыми и пробелами до температуры, так как этот текст нам не нужен, в скобки его не заключаем – это не группа);
* `\s`: перед температурой днем обязательно идет пробел (тоже не интересен, не группа);
* `(-?\d+°)`: группа для температуры днем – набор цифр с возможным минусом в начале;
* `.+\s`: снова набор любых символов и пробел (не группа);
* `(-?\d+°)`: группа для температуры ночью – набор цифр с возможным минусом в начале.

Другими словами, мы написали очень подробный шаблон, по которому устроены строки с прогнозами погоды, и теперь можем из результатов поиска по этому шаблону извлекать отдельные фрагменты, которые нас интересуют. 

Так как пример здесь специально подобран простой, кажется, что и без регулярных выражений можно обойтись, разбив прогнозы по запятой. Это так, но в реальной жизни не всегда текст представлен в унифицированном формате. Если бы в качестве разделителя могли выступать и запятые, и точки с запятой (где как, база данных, собранная вручную разными людьми, например), то группы бы очень помогли. 

**Дополнение.** В `pandas` в подмодуле `str` для строк есть метод `extract()`, который поддерживает регулярные выражения, с группами в том числе. Представим, что вместо списка строк `pogoda` есть датафрейм из одного текстового столбца:

In [51]:
import pandas as pd

df = pd.DataFrame(pogoda, columns = ["pogoda"])
df

,pogoda
0,"вчера, 11 февраля, небольшой дождь, днём 10°, ..."
1,"сегодня, 12 февраля, малооблачно, днём 10°, но..."
2,"четверг, 13 февраля, облачно с прояснениями, д..."
3,"пятница, 14 февраля, пасмурно, днём 7°, ночью 5°"
4,"суббота, 15 февраля, ливень, днём 8°, ночью 6°"
5,"сегодня, 11 февраля, пасмурно, днём -4°, ночью..."
6,"среда, 12 февраля, пасмурно, днём -4°, ночью -6°"
7,"четверг, 13 февраля, пасмурно, днём -5°, ночью..."
8,"пятница, 14 февраля, небольшой снег, днём -4°,..."
9,"суббота, 15 февраля, небольшой снег, днём -6°,..."


Впишем в `str.extract()` выражение с группами выше:

In [52]:
df["pogoda"].str.extract("(\d+)\s([а-я]+).+\s(-?\d+°).+\s(-?\d+°)")

,0,1,2,3
0,11,февраля,10°,2°
1,12,февраля,10°,4°
2,13,февраля,10°,5°
3,14,февраля,7°,5°
4,15,февраля,8°,6°
5,11,февраля,-4°,-6°
6,12,февраля,-4°,-6°
7,13,февраля,-5°,-5°
8,14,февраля,-4°,-6°
9,15,февраля,-6°,-10°


Метод извлек фрагменты, соответствующие группам, выделенным в выражении, и сохранил каждую группу в отдельный столбец!

In [53]:
df_new = df["pogoda"].str.extract("(\d+)\s([а-я]+).+\s(-?\d+°).+\s(-?\d+°)")
df_new.columns = ["day", "month", "temp_d", "temp_n"]
df_new

,day,month,temp_d,temp_n
0,11,февраля,10°,2°
1,12,февраля,10°,4°
2,13,февраля,10°,5°
3,14,февраля,7°,5°
4,15,февраля,8°,6°
5,11,февраля,-4°,-6°
6,12,февраля,-4°,-6°
7,13,февраля,-5°,-5°
8,14,февраля,-4°,-6°
9,15,февраля,-6°,-10°
